In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install emoji
! pip install ekphrasis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import nltk
import emoji
from collections import Counter
from itertools import groupby

DATA_DIR = './development_i/'
CORPUS_DIR = '/content/drive/Shareddrives/HLT/dataset/'
RANDOM_STATE = 211299

In [4]:
with open(CORPUS_DIR + 'paisa.raw.utf8', 'r') as file:
    paisa_corpus = file.readlines()

In [5]:
len(paisa_corpus)

8332059

### Elimino tutte le stringhe strane (poco utili)

In [6]:
def get_chars_in_range(start_c, end_c):
  start_ascii = ord(start_c)
  end_ascii = ord(end_c)
  return [chr(i) for i in range(start_ascii, end_ascii + 1)]

In [7]:
admissible_chars = []
admissible_chars.extend(get_chars_in_range('A', 'Z'))
admissible_chars.extend(get_chars_in_range('a', 'z'))
admissible_chars.extend(get_chars_in_range('0', '9'))
admissible_chars.extend(['È', 'è'])

In [8]:
paisa_corpus = [item for item in paisa_corpus if item[0] in admissible_chars]

In [9]:
len(paisa_corpus)

7334309

In [10]:
correct_words = ['alcool', 'cool', 'preesame']

def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text)
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' affan[xx**]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' ca\*\*o ', ' cazzo ', text, flags=re.IGNORECASE)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' cazxxx ', ' cazzate ', text)
    text = re.sub(r' CAZO ', ' CAZZO ', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m\*\*\*a ', ' merda ', text, flags=re.IGNORECASE)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' culx ', ' culo ', text)
    text = re.sub(r' p[u.x*@%#$^]+ana ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' stronz[.x*@%#$^]+a ', ' stronzata ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    text = re.sub(r' min[*t]+a ', ' minchiata ', text)
    return text

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in correct_words):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

def preprocess_string(string):
    # Remove newlines and carriage returns
    string = string.replace('\n', ' ').replace('\r', ' ')

    # Remove incorrect bad word
    string = clean_disguised_bad_words(string)

    # Remove duplicates
    string = deleteDuplicate(string)

    # Replace emojis with text
    string = emoji.demojize(string, language='it')

    # Remove special characters and numbers, preserve placeholders
    processed_tweet = re.sub(r'[^a-zA-ZÀ-ú\d\U00010000-\U0010ffff]', ' ', string)

    return processed_tweet

In [11]:
paisa_corpus = [preprocess_string(item) for item in paisa_corpus]

In [14]:
with open('/content/drive/Shareddrives/HLT/dataset/italian_corpus.txt') as f: # già preprocessato
  tweet_corpus = f.readlines()

tweet_corpus = [re.sub(r'\s+', ' ', string) for string in tweet_corpus]

In [12]:
paisa_corpus = [re.sub(r'\s+', ' ', string) for string in paisa_corpus]
paisa_corpus_df = pd.DataFrame(paisa_corpus)

paisa_corpus_df

,0
0,Davide Guglielmini finito agli arresti domicil...
1,Avete partecipato in tanti alla bella discussi...
2,Otto mesi di ritardo ma si parte ecco la novit...
3,Il prezzo dei biglietti è fissato a 7 non è pr...
4,Dopo la Shopping experience di Coin arriva la ...
...,...
7334304,Psichiatria
7334305,La psichiatria è la branca specialistica della...
7334306,Henry Ey 1900 1977 psichiatra francese scrive ...
7334307,La psichiatria è una pratica medica focalizzat...


In [13]:
paisa_corpus_df.to_csv('/content/drive/Shareddrives/HLT/dataset/paisa_processed.txt', header=False, sep='\t', index=False)

### Paisa corpus

In [ ]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/paisa.raw.utf8 --name paisa --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/paisa.raw.utf8
100% 8332059/8332059 [09:41<00:00, 14316.62it/s]

Writing 1-grams...
entries:1,222,294	-	tokens:215,357,765
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/paisa/counts_1grams.txt
Writing 2-grams...
entries:26,915,158	-	tokens:215,357,765
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/paisa/counts_2grams.txt


In [ ]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/paisa")
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonifuoridaicoglioni"))
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonipiaga"))
print(seg.segment("salvinipagliaccio"))
print(seg.segment("governodeipeggiori"))
print(seg.segment("speranzadimettiti"))
print(seg.segment("speranzavattene"))
print(seg.segment("m5s"))
print(seg.segment("salviniportasfiga"))
print(seg.segment("lega"))
print(seg.segment("l'egosmisuratodeitovagliolini"))
print(seg.segment("orcaputtanachesvitato"))
print(seg.segment("nnciferma+nessuno"))
print(seg.segment("slavinistupidone"))
print(seg.segment("slavinostupidone"))
print(seg.segment("salvinistupidone"))

Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/paisa - 1grams ...
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/paisa - 2grams ...
salvini sciacallo
meloni fuori dai coglioni
salvini sciacallo
meloni piaga
salvini pagliaccio
governo dei peggiori
speranza di mettiti
speranza vattene
m5s
salvini porta sfiga
lega
l'ego smisurato dei to vaglio lin i
orca puttana che s vita to
nnciferma+ nessuno
s la vini stupido ne
slavi no stupido ne
salvini stupido ne


confronto con italian corpus

In [ ]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt --name tweet_ita --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt
100% 10013615/10013615 [06:51<00:00, 24362.61it/s]

Writing 1-grams...
entries:326,541	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_1grams.txt
Writing 2-grams...
entries:5,653,502	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_2grams.txt


In [27]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita")
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonifuoridaicoglioni"))
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonipiaga"))
print(seg.segment("salvinipagliaccio"))
print(seg.segment("governodeipeggiori"))
print(seg.segment("speranzadimettiti"))
print(seg.segment("speranzavattene"))
print(seg.segment("m5s"))
print(seg.segment("salviniportasfiga"))
print(seg.segment("lega"))
print(seg.segment("l'egosmisuratodeitovagliolini"))
print(seg.segment("orcaputtanachesvitato"))
print(seg.segment("nnciferma+nessuno"))

Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...
stats file not available!


SystemExit: 1

### Corpus mergiato

In [15]:
extended_corpus = tweet_corpus + paisa_corpus # concateno i due corpus differenti

In [16]:
extended_corpus_df = pd.DataFrame(extended_corpus)
extended_corpus_df.to_csv('/content/drive/Shareddrives/HLT/dataset/extended.txt', header=False, sep='\t', index=False)

In [23]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/extended.txt --name extended --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/extended.txt
100% 24682233/24682233 [16:52<00:00, 24378.38it/s]

Writing 1-grams...
entries:1,331,682	-	tokens:380,706,261
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/extended/counts_1grams.txt
Writing 2-grams...
entries:29,843,958	-	tokens:380,706,261
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/extended/counts_2grams.txt


In [30]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/extended")
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonifuoridaicoglioni"))
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonipiaga"))
print(seg.segment("salvinipagliaccio"))
print(seg.segment("governodeipeggiori"))
print(seg.segment("speranzadimettiti"))
print(seg.segment("speranzavattene"))
print(seg.segment("m5s"))
print(seg.segment("salviniportasfiga"))
print(seg.segment("lega"))
print(seg.segment("l'egosmisuratodeitovagliolo"))
print(seg.segment("orcaputtanachesvitato"))
print(seg.segment("nnciferma+nessuno"))
print(seg.segment("facciorete"))

Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/extended - 1grams ...
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/extended - 2grams ...
salvini sciacallo
meloni fuori dai coglioni
salvini sciacallo
meloni piaga
salvini pagliaccio
governo dei peggiori
speranza dimettiti
speranza vattene
m5s
salvini porta sfiga
lega
l'ego smisurato dei tovagliolo
orca puttana che s vita to
nnciferma+ nessuno
faccio rete


In [17]:
print(extended_corpus_df.iloc[17347921])

0    Henry Ey 1900 1977 psichiatra francese scrive ...
Name: 17347921, dtype: object
